In [ ]:
# I have two helper scripts in python because I could not get it to work in R.
# In some cases of evaluation and correlation, I had to use chatgpt because I could not get it to work and I still did not meet the deadline.

In [146]:
# Prepare an environment for Python scripts (some functionalities were implemented in python due to problems with their R alternatives)
invisible(system("python3 -m venv .venv_py", intern = TRUE))
invisible(system(".venv_py/bin/pip install geopandas contextily matplotlib pandas", intern = TRUE))


In [1]:
# Install and load required R packages
packages <- c("dplyr","readr","cluster","proxy","ggplot2","viridis","reshape2","pheatmap","tibble","stringr","ggpubr","gridExtra")
install.packages(setdiff(packages, rownames(installed.packages())))
lapply(packages, library, character.only = TRUE)

library(dplyr)
library(readr)
library(cluster)
library(proxy)
library(ggplot2)
library(viridis)
library(reshape2)
library(pheatmap)
library(tibble)
library(stringr)
library(ggpubr)
library(gridExtra)



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘proxy’


The following objects are masked from ‘package:stats’:

    as.dist, dist


The following object is masked from ‘package:base’:

    as.matrix


Loading required package: viridisLite


Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine




[[1]]
[1] "dplyr"     "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[7] "methods"   "base"     

[[2]]
[1] "readr"     "dplyr"     "stats"     "graphics"  "grDevices" "utils"    
[7] "datasets"  "methods"   "base"     

[[3]]
 [1] "cluster"   "readr"     "dplyr"     "stats"     "graphics"  "grDevices"
 [7] "utils"     "datasets"  "methods"   "base"     

[[4]]
 [1] "proxy"     "cluster"   "readr"     "dplyr"     "stats"     "graphics" 
 [7] "grDevices" "utils"     "datasets"  "methods"   "base"     

[[5]]
 [1] "ggplot2"   "proxy"     "cluster"   "readr"     "dplyr"     "stats"    
 [7] "graphics"  "grDevices" "utils"     "datasets"  "methods"   "base"     

[[6]]
 [1] "viridis"     "viridisLite" "ggplot2"     "proxy"       "cluster"    
 [6] "readr"       "dplyr"       "stats"       "graphics"    "grDevices"  
[11] "utils"       "datasets"    "methods"     "base"       

[[7]]
 [1] "reshape2"    "viridis"     "viridisLite" "ggplot2"     "proxy"      
 [6] "cluster"     "readr"       "dplyr"       "stats"       "graphics"   
[11] "grDevices"   "utils"       "datasets"    "methods"     "base"       

[[8]]
 [1] "pheatmap"    "reshape2"    "viridis"     "viridisLite" "ggplot2"    
 [6] "proxy"       "cluster"     "readr"       "dplyr"       "stats"      
[11] "graphics"    "grDevices"   "utils"       "datasets"    "methods"    
[16] "base"       

[[9]]
 [1] "tibble"      "pheatmap"    "reshape2"    "viridis"     "viridisLite"
 [6] "ggplot2"     "proxy"       "cluster"     "readr"       "dplyr"      
[11] "stats"       "graphics"    "grDevices"   "utils"       "datasets"   
[16] "methods"     "base"       

[[10]]
 [1] "stringr"     "tibble"      "pheatmap"    "reshape2"    "viridis"    
 [6] "viridisLite" "ggplot2"     "proxy"       "cluster"     "readr"      
[11] "dplyr"       "stats"       "graphics"    "grDevices"   "utils"      
[16] "datasets"    "methods"     "base"       

[[11]]
 [1] "ggpubr"      "stringr"     "tibble"      "pheatmap"    "reshape2"   
 [6] "viridis"     "viridisLite" "ggplot2"     "proxy"       "cluster"    
[11] "readr"       "dplyr"       "stats"       "graphics"    "grDevices"  
[16] "utils"       "datasets"    "methods"     "base"       

[[12]]
 [1] "gridExtra"   "ggpubr"      "stringr"     "tibble"      "pheatmap"   
 [6] "reshape2"    "viridis"     "viridisLite" "ggplot2"     "proxy"      
[11] "cluster"     "readr"       "dplyr"       "stats"       "graphics"   
[16] "grDevices"   "utils"       "datasets"    "methods"     "base"

In [148]:
# This script processes a CSV file containing ASV data and clusters the sequences into OTUs using VSEARCH.

# Data provided by authors
fasta_path <- "initial_data/seqtable_readyforanalysis.fasta"

uc_path <- "computed_data/otu97.uc"
centroids_path <- "computed_data/otu97.fasta"

# VSEARCH clustering with 97% identity treshold
vsearch_cmd <- paste(
  "vsearch",
  "--cluster_fast", shQuote(fasta_path),
  "--id 0.97",
  "--centroids", shQuote(centroids_path),
  "--uc", shQuote(uc_path)
)

result <- system(vsearch_cmd, intern = TRUE, ignore.stderr = FALSE)
cat(result, sep = "\n")


In [149]:
# This script processes the output of VSEARCH to create an OTU table from the ASV data. For now, the OTUs are not taxonomically annotated.

system(".venv_py/bin/python generate_otu_table.py", intern = TRUE)

[1] "Saved OTU table with shape (2210, 255) → computed_data/otu_table_97.csv"

In [150]:
# This script adds taxonomy information to the OTUs based on data provided by authors.

taxa_path     <- "initial_data/taxa_wsp_readyforanalysis.csv"
uc_path       <- "computed_data/otu97.uc"
taxonomy_path <- "computed_data/taxonomy_otu_97.csv"


uc <- read_tsv(uc_path, col_names = FALSE, comment = "#", show_col_types = FALSE) |>
  filter(X1 %in% c("H", "S")) |> # hits (H) and seeds (S)
  transmute(ASV = X10, OTU = paste0("OTU_", X2))

# Read taxonomy supplied by authors and join it with ASV-to-OTU map
taxa_df <- read_tsv(taxa_path, show_col_types = FALSE) |>
  rename(ASV = 1)

merged <- inner_join(uc, taxa_df, by = "ASV")
tax_cols <- c("Kingdom", "Phylum", "Class", "Order", "Family", "Genus", "Species")

# Check what is the most common taxonomy for each OTU
most_common <- \(x) {
  x <- x[!is.na(x) & x != ""]
  if (length(x) == 0) NA_character_ else names(sort(table(x), TRUE))[1]
}

taxonomy_df <- merged |>
  group_by(OTU) |>
  summarise(across(all_of(tax_cols), most_common), .groups = "drop")

write_csv(taxonomy_df, taxonomy_path)
cat("Added taxonomy information to OTUs\n")


Added taxonomy information to OTUs


In [ ]:
# PCA and k-means clustering function for paired 0D vs 7D samples

cluster_kmeans_pca <- function(k_init, k_final) {
  
  set.seed(42)                                                      # reproducible
  
  meta <- read.delim("initial_data/metadata_Time0D-7D-4M_May2022_wJSDpart-merged.csv", check.names = FALSE)
  otu  <- read.csv("computed_data/otu_table_97.csv", row.names = 1, check.names = FALSE)
  
  # filter to keep 0D and 7D replicates only (there are some reduntant data)
  meta <- meta %>%
    filter(Experiment %in% c("0D", paste0("7D_rep", 1:4)),
           !is.na(parent))
  
  init_samples  <- meta %>% filter(Experiment == "0D")
  final_samples <- meta %>% filter(Experiment != "0D")
  
  common_parent <- intersect(init_samples$parent, final_samples$parent)
  init_samples  <- init_samples  %>% filter(parent %in% common_parent)
  final_samples <- final_samples %>% filter(parent %in% common_parent)
  
  pairs <- inner_join(
    init_samples[,  c("sampleid", "parent")],
    final_samples[, c("sampleid", "parent")],
    by = "parent",
    suffix = c("_init", "_final")
  )
  
  # OTU table subset and filters
  samples_use <- c(pairs$sampleid_init, pairs$sampleid_final)
  otu_sub     <- otu[samples_use, , drop = FALSE]
  
  present  <- colSums(otu_sub > 0) >= 0.05 * nrow(otu_sub) # presence in at least 5% of samples
  if (!any(present)) stop("No OTUs pass presence filter.")
  otu_sub <- otu_sub[, present]
  
  variable <- apply(otu_sub, 2, var) > 1e-6     # variance filter
  otu_sub <- otu_sub[, variable]
  
  # transform + PCA
  scores <- prcomp(scale(log1p(otu_sub)), center = TRUE)$x[, 1:min(10, ncol(otu_sub)), drop = FALSE]
  
  init_scores  <- scores[pairs$sampleid_init,  , drop = FALSE]
  final_scores <- scores[pairs$sampleid_final, , drop = FALSE]
  
  km_init  <- kmeans(init_scores,  centers = k_init)
  km_final <- kmeans(final_scores, centers = k_final)
  
  out_df <- bind_rows(
    data.frame(sampleid = pairs$sampleid_init,  parent = pairs$parent, timepoint = "initial", cluster = km_init$cluster),
    data.frame(sampleid = pairs$sampleid_final, parent = pairs$parent, timepoint = "final",   cluster = km_final$cluster)
  )
  
  dir.create("computed_data", showWarnings = FALSE)
  out_path <- sprintf("computed_data/otu_cluster_assignments_KMEANS_PCA_%dx%d.csv", k_init, k_final)
  write.csv(out_df, out_path, row.names = FALSE)
  cat("Saved:", out_path, "\n")
}


In [152]:
# jsd and PAM clustering function for paired 0D vs 7D samples
cluster_pam_jsd <- function(k_init = 5, k_final = 5, output_dir = "computed_data") {
  
  meta <- read.delim("initial_data/metadata_Time0D-7D-4M_May2022_wJSDpart-merged.csv", check.names = FALSE)
  otu <- read.csv("computed_data/otu_table_97.csv", row.names = 1, check.names = FALSE)
  

  meta <- meta %>%
    filter(Experiment %in% c("0D", paste0("7D_rep", 1:4)), !is.na(parent))
  
  init <- meta %>% filter(Experiment == "0D")
  final <- meta %>% filter(Experiment != "0D")
  
  common_parent <- intersect(init$parent, final$parent)
  init <- init  %>% filter(parent %in% common_parent)
  final <- final %>% filter(parent %in% common_parent)
  
  pairs <- inner_join(
    init[ ,c("sampleid", "parent")],
    final[,c("sampleid", "parent")],
    by = "parent",
    suffix = c("_init", "_final")
  )
  
  # relative abundances
  samples <- c(pairs$sampleid_init, pairs$sampleid_final)
  otu_rel <- otu[samples, , drop = FALSE] / rowSums(otu[samples, ])
  
  # Jensen-Shannon distance
  jsd <- function(x, y) {
    x <- ifelse(x == 0, 1e-10, x)
    y <- ifelse(y == 0, 1e-10, y)
    m <- (x + y) / 2
    0.5 * sum(x * log2(x / m)) + 0.5 * sum(y * log2(y / m))
  }
  
  d_all <- proxy::dist(otu_rel, method = jsd)
  d_mat <- as.matrix(d_all)
  
  d_init <- as.dist(d_mat[pairs$sampleid_init,  pairs$sampleid_init])
  d_final <- as.dist(d_mat[pairs$sampleid_final, pairs$sampleid_final])
  
  pam_i <- cluster::pam(d_init,  k = k_init)
  pam_f <- cluster::pam(d_final, k = k_final)
  
  out_df <- bind_rows(
    data.frame(sampleid = pairs$sampleid_init,  parent = pairs$parent, timepoint = "initial", cluster = pam_i$clustering),
    data.frame(sampleid = pairs$sampleid_final, parent = pairs$parent, timepoint = "final",   cluster = pam_f$clustering)
  )
  
  dir.create(output_dir, showWarnings = FALSE)
  out_path <- sprintf("%s/otu_cluster_assignments_PAM_JSD_%dx%d.csv", output_dir, k_init, k_final)
  write.csv(out_df, out_path, row.names = FALSE)
  cat("Saved:", out_path, "\n")
  
  invisible(out_df)
}


In [153]:
# SOme clustering evaluation functions
# These functions evaluate clustering results using silhouette width and internal criteria like Calinski-Harabasz
# and Davies-Bouldin indices. They require the clustering results and the relative abundance OTU
# Calinski-Harabasz was also used by original autors.

evaluate_clusters <- function(assign_df, otu_rel_mat) {
  if (is.null(assign_df)) return(NULL)
  
  assign_df <- dplyr::arrange(assign_df, sampleid)
  otu_rel_mat <- otu_rel_mat[assign_df$sampleid, , drop = FALSE]
  
  keep <- apply(otu_rel_mat, 2, var) > 1e-6
  otu_rel_mat <- otu_rel_mat[, keep, drop = FALSE]
  otu_rel_mat[is.na(otu_rel_mat)] <- 0
  
  clusters <- as.integer(assign_df$cluster)
  
  sil <- suppressWarnings(
    mean(cluster::silhouette(clusters, dist(otu_rel_mat))[, 3])
  )
  
  idx <- clusterCrit::intCriteria(as.matrix(otu_rel_mat), clusters, c("Calinski_Harabasz", "Davies_Bouldin"))
  
  tibble::tibble(
    silhouette        = sil,
    calinski_harabasz = idx$calinski_harabasz,
    davies_bouldin    = idx$davies_bouldin
  )
}

In [ ]:
# This scripts actually checks different methods and parameters of initial and final cluster numbers

# mettircs
evaluate_clusters <- function(df, otu_rel) { 
  if (is.null(df)) return(NULL)
  df <- df[order(df$sampleid), ]
  mat <- otu_rel[df$sampleid, , drop = FALSE]
  mat <- mat[, apply(mat, 2, var) > 1e-6, drop = FALSE]
  mat[is.na(mat)] <- 0
  cl  <- as.integer(df$cluster)
  sil <- tryCatch(mean(cluster::silhouette(cl, dist(mat))[, 3]), error = \(e) NA)
  idx <- tryCatch(clusterCrit::intCriteria(as.matrix(mat), cl, c("Calinski_Harabasz", "Davies_Bouldin")),
                  error = \(e) list(calinski_harabasz = NA, davies_bouldin = NA))
  tibble(silhouette = sil, calinski_harabasz = idx$calinski_harabasz, davies_bouldin = idx$davies_bouldin)
}

# grid search amongst 2-6 initial clusters and 2-6 final clusters
grid_search_clustering <- function(k_range = 2:6, out_dir = "computed_data/clusterings") {
  otu_path  <- "computed_data/otu_table_97.csv"
  meta_path <- "initial_data/metadata_Time0D-7D-4M_May2022_wJSDpart-merged.csv"
  
  otu_rel <- sweep(read.csv(otu_path, row.names = 1, check.names = FALSE), 1,
                   rowSums(read.csv(otu_path, row.names = 1, check.names = FALSE)), "/")
  
  meta <- read.delim(meta_path) %>% filter(Experiment %in% c("0D", paste0("7D_rep", 1:4)), !is.na(parent))
  init <- meta %>% filter(Experiment == "0D")
  final <- meta %>% filter(Experiment != "0D")
  common <- intersect(init$parent, final$parent)
  init  <- init  %>% filter(parent %in% common)
  final <- final %>% filter(parent %in% common)
  
  pairs <- inner_join(init[, c("sampleid", "parent")], final[, c("sampleid", "parent")],
                      by = "parent", suffix = c("_init", "_final"))
  
  otu_rel <- otu_rel[c(pairs$sampleid_init, pairs$sampleid_final), , drop = FALSE]
  dir.create(out_dir, recursive = TRUE, showWarnings = FALSE)
  
  res <- list()
  for (k_i in k_range) for (k_f in k_range) {
    f_km  <- sprintf("%s/otu_cluster_assignments_KMEANS_PCA_%dx%d.csv", out_dir, k_i, k_f)
    f_pam <- sprintf("%s/otu_cluster_assignments_PAM_JSD_%dx%d.csv",   out_dir, k_i, k_f)
    # we only run clustering if the results are not already computed
    df_km  <- if (file.exists(f_km))  read_csv(f_km,  show_col_types = FALSE) else {x <- cluster_kmeans_pca(k_i, k_f); write.csv(x, f_km,  row.names = FALSE); x}
    df_pam <- if (file.exists(f_pam)) read_csv(f_pam, show_col_types = FALSE) else {y <- cluster_pam_jsd(k_i, k_f);  write.csv(y, f_pam, row.names = FALSE); y}
    
    mk <- evaluate_clusters(df_km,  otu_rel)
    mp <- evaluate_clusters(df_pam, otu_rel)
    if (!is.null(mk)) res <- append(res, list(mutate(mk, method = "KMEANS-PCA", k_init = k_i, k_final = k_f)))
    if (!is.null(mp)) res <- append(res, list(mutate(mp, method = "PAM-JSD",   k_init = k_i, k_final = k_f)))
  }
  
  summary_df <- bind_rows(res)
  write.csv(summary_df, "computed_data/clustering_evaluation_summary.csv", row.names = FALSE)
  summary_df %>% arrange(desc(silhouette))
}


In [155]:
# This may take a while (up to ~35min), depending on the range of k values and already present files. I left them in so it takes more like 2 minutes.
#
grid_search_clustering()

# The best clustering method and parameters pair was the PAM-JSD method with k_init = 5 and k_final = 2. - same as original authors.

silhouette,calinski_harabasz,davies_bouldin,method,k_init,k_final
<dbl>,<dbl>,<dbl>,<chr>,<int>,<int>
6.737007e-02,298.36367,2.076623,PAM-JSD,5,2
5.910104e-02,261.40550,2.283779,PAM-JSD,6,2
5.660027e-02,140.87712,3.496999,KMEANS-PCA,2,2
4.798709e-02,53.59509,6.276912,KMEANS-PCA,3,2
4.038125e-02,170.88727,5.023777,PAM-JSD,5,4
3.911207e-02,252.98454,2.483951,PAM-JSD,6,3
3.596206e-02,266.30617,2.158059,PAM-JSD,4,2
3.591069e-02,95.07587,8.052514,PAM-JSD,3,2
3.417088e-02,99.41988,6.203004,PAM-JSD,4,3


In [ ]:
# this script generates a heatmap of cluster transitions between 0D and 7D samples (initial to final)

cluster_file <- "computed_data/clusterings/otu_cluster_assignments_PAM_JSD_5x2.csv"
# Choose TRUE to normalize the counts
do_norm <- FALSE

meta <- read.delim("initial_data/metadata_Time0D-7D-4M_May2022_wJSDpart-merged.csv") %>%
  filter(Experiment %in% c("0D", paste0("7D_rep", 1:4)), !is.na(parent)) %>%
  mutate(sampleid = sub("^X", "", sampleid))

clusters <- read_csv(cluster_file, show_col_types = FALSE) %>%
  mutate(sampleid = sub("^X", "", sampleid))

pairs <- inner_join(filter(meta, Experiment == "0D")  %>% select(sampleid, parent),
                    filter(meta, Experiment != "0D") %>% select(sampleid, parent),
                    by = "parent", suffix = c("_init", "_final")) %>%
  left_join(filter(clusters, timepoint == "initial") %>% rename(clust_init = cluster),
            by = c("sampleid_init" = "sampleid")) %>%
  left_join(filter(clusters, timepoint == "final")   %>% rename(clust_final = cluster),
            by = c("sampleid_final" = "sampleid")) %>%
  filter(!is.na(clust_init), !is.na(clust_final))

tab <- with(pairs, table(clust_init, clust_final))
if (do_norm) tab <- sweep(tab, 1, rowSums(tab), "/")

df <- as.data.frame(tab) %>% rename(Initial = clust_init, Final = clust_final, Value = Freq)

gg <- ggplot(df, aes(Final, Initial, fill = Value)) +
  geom_tile(colour = "white") +
  geom_text(aes(label = if (do_norm) sprintf("%.2f", Value) else Value), size = 4) +
  scale_fill_viridis_c(option = if (do_norm) "viridis" else "magma",
                       begin = .2, end = 1,
                       name = if (do_norm) "Fraction" else "Count") +
  theme_minimal() +
  labs(title = if (do_norm) "Normalised cluster transitions (0D → 7D)" else "Cluster transitions (0D → 7D)",
       x = "Final cluster", y = "Initial cluster")

ggsave(sprintf("plots/otu_cluster_assignments_PAM_JSD_5x2%s.png",
               if (do_norm) "_heatmap_normalised" else "_heatmap"),
       gg, width = 10, height = 6, dpi = 300)


Warning message in left_join(., filter(clusters, timepoint == "initial") %>% rename(clust_init = cluster), :
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 1 of `x` matches multiple rows in `y`.
ℹ Row 1 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship =
  "many-to-many"` to silence this warning.”


In [165]:
# This script generates a heat-map of cluster transitions from 0D to 7D samples, showing how clusters change over time


clust_file <- "computed_data/clusterings/otu_cluster_assignments_PAM_JSD_5x2.csv"
meta_file  <- "initial_data/metadata_Time0D-7D-4M_May2022_wJSDpart-merged.csv"
# Choose TRUE to normalize the counts
do_norm    <- TRUE

clusters <- read_csv(clust_file, show_col_types = FALSE)
meta     <- read.delim(meta_file)

clusters <- clusters %>% filter(timepoint %in% c("initial", "final"))
clusters$sampleid <- sub("^X", "", clusters$sampleid)
meta$sampleid     <- sub("^X", "", meta$sampleid)

clusters <- left_join(clusters, meta[, c("sampleid", "Location")], by = "sampleid") %>%
  filter(!is.na(Location))

clust_i <- clusters %>%
  filter(timepoint == "initial") %>%
  select(sampleid, cluster, Location, parent) %>%
  rename(cluster_init = cluster, forest = Location)

clust_f <- clusters %>%
  filter(timepoint == "final") %>%
  select(sampleid, cluster, parent) %>%
  rename(cluster_final = cluster)

pairs <- inner_join(clust_i, clust_f, by = "parent")

tab_all <- pairs %>%
  group_by(forest, cluster_init, cluster_final) %>%
  summarise(count = n(), .groups = "drop")

if (do_norm) {
  tab_all <- tab_all %>%
    group_by(forest, cluster_init) %>%
    mutate(fraction = count / sum(count)) %>%
    ungroup()
}

gg <- ggplot(tab_all, aes(
  x = factor(cluster_final),
  y = factor(cluster_init),
  fill = if (do_norm) fraction else count
)) +
  geom_tile(color = "white") +
  geom_text(aes(label = if (do_norm) sprintf("%.2f", ifelse(is.na(fraction), 0, fraction)) else count), size = 3) +
  scale_fill_viridis_c(option = "viridis", begin = 0.2, end = 1) +
  facet_wrap(~ forest) +
  theme_minimal() +
  theme(
    panel.background = element_rect(fill = "white", color = NA),
    plot.background = element_rect(fill = "white", color = NA)
  ) +
  labs(
    title = if (do_norm) "Cluster transitions per forest (normalised)"
            else "Cluster transitions per forest (counts)",
    x = "Final cluster",
    y = "Initial cluster",
    fill = if (do_norm) "Fraction" else "Count"
  )

tag <- if (do_norm) "normalised" else "counts"
plot_path <- sprintf("plots/otu_cluster_assignments_PAM_JSD_5x2_per_location_%s.png", tag)
ggsave(plot_path, gg, width = 14, height = 8, dpi = 300, bg = "white")
cat("Saved plot:", plot_path, "\n"); flush.console()


Warning message in inner_join(clust_i, clust_f, by = "parent"):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 1 of `x` matches multiple rows in `y`.
ℹ Row 1 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship =
  "many-to-many"` to silence this warning.”


Saved plot: plots/otu_cluster_assignments_PAM_JSD_5x2_per_location_normalised.png 


In [ ]:
# This script generates a heatmap of the 25 most variable OTUs across clusters

clust_file <- "computed_data/clusterings/otu_cluster_assignments_PAM_JSD_5x2.csv"
otu        <- read.csv("computed_data/otu_table_97.csv", row.names = 1, check.names = FALSE)
taxa       <- read.csv("computed_data/taxonomy_otu_97.csv",            check.names = FALSE)
clusters   <- read_csv(clust_file, show_col_types = FALSE)

# gather IDs and keep overlap
clusters$sampleid <- sub("^X", "", clusters$sampleid)
rownames(otu)     <- sub("^X", "", rownames(otu))
common            <- intersect(clusters$sampleid, rownames(otu))
otu               <- otu[common, , drop = FALSE]
clusters          <- clusters[match(rownames(otu), clusters$sampleid), ]

#relative abundance and cluster labels
otu_rel <- sweep(otu, 1, rowSums(otu), "/")
if ("timepoint" %in% names(clusters)) clusters$cluster <- paste0(clusters$timepoint, "_", clusters$cluster)

otu_long <- melt(cbind(sampleid = rownames(otu_rel), cluster = clusters$cluster, otu_rel),
                 id.vars = c("sampleid", "cluster"),
                 variable.name = "otu", value.name = "abund")

otu_summary <- otu_long %>%
  group_by(cluster, otu) %>%
  summarise(mean_abund = mean(abund, na.rm = TRUE), .groups = "drop") %>%
  tidyr::pivot_wider(names_from = cluster, values_from = mean_abund)

#top-25 by variance
var_vec <- apply(select(otu_summary, -otu), 1, var, na.rm = TRUE)
top <- otu_summary %>% slice_max(var_vec, n = 25)

mat <- select(top, -otu) %>% as.matrix()
rownames(mat) <- top$otu
storage.mode(mat) <- "numeric"

# add taxonomy labels
label_df <- taxa %>% mutate(label = ifelse(!is.na(Genus)   & Genus   != "", Genus,
                                           ifelse(!is.na(Species) & Species != "", Species, OTU))) %>% select(OTU, label)
rowlab <- tibble::deframe(label_df)[rownames(mat)]
rowlab[is.na(rowlab)] <- rownames(mat)[is.na(rowlab)]
dup <- duplicated(rowlab); rowlab[dup] <- paste0(rowlab[dup], "_", seq_len(sum(dup)))
rownames(mat) <- rowlab

mat <- mat[, sort(colnames(mat), method = "radix")]

out_path <- file.path("plots", sub("\\.csv$", "_top25_otus.png", basename(clust_file)))
pheatmap(mat, scale = "row", cluster_cols = FALSE, cluster_rows = TRUE,
         angle_col = 45, fontsize_row = 7, fontsize_col = 10, filename = out_path)
cat("Saved:", out_path, "\n")


Saved: plots/otu_cluster_assignments_PAM_JSD_5x2_top25_otus.png 


In [ ]:
# This script generates a heatmap of the top 25 OTUs per trajectory (initial to final clusters)

clust_file <- "computed_data/clusterings/otu_cluster_assignments_PAM_JSD_5x2.csv"

otu <- read.csv("computed_data/otu_table_97.csv", row.names = 1, check.names = FALSE); rownames(otu) <- sub("^X","",rownames(otu))
taxonomy <- read.csv("computed_data/taxonomy_otu_97.csv", check.names = FALSE)
clusters <- readr::read_csv(clust_file, show_col_types = FALSE) %>% dplyr::filter(timepoint %in% c("initial","final")) %>% dplyr::mutate(sampleid=sub("^X","",sampleid))

clust_i <- clusters %>% dplyr::filter(timepoint=="initial") %>% dplyr::select(sampleid,parent,cluster_init=cluster)
clust_f <- clusters %>% dplyr::filter(timepoint=="final")   %>% dplyr::select(sampleid,parent,cluster_final=cluster)
pairs <- dplyr::inner_join(clust_i,clust_f,by="parent") %>% dplyr::mutate(traj=paste0("init",cluster_init,"_to_fin",cluster_final))

samples <- unique(c(pairs$sampleid.x,pairs$sampleid.y))
otu_rel <- sweep(otu[samples,,drop=FALSE],1,rowSums(otu[samples,,drop=FALSE]),"/") %>% tibble::rownames_to_column("sampleid")
traj_lab <- dplyr::bind_rows(pairs %>% dplyr::select(sampleid=sampleid.x,traj), pairs %>% dplyr::select(sampleid=sampleid.y,traj))
otu_labeled <- dplyr::left_join(traj_lab,otu_rel,by="sampleid")

otu_summary <- reshape2::melt(otu_labeled,id.vars=c("sampleid","traj"),variable.name="otu",value.name="abund") %>% 
               dplyr::group_by(traj,otu) %>% dplyr::summarise(mean_abund=mean(abund,na.rm=TRUE),.groups="drop") %>% 
               tidyr::pivot_wider(names_from=traj,values_from=mean_abund)

var_vec <- apply(dplyr::select(otu_summary,-otu),1,var,na.rm=TRUE)
top  <- otu_summary %>% dplyr::slice_max(var_vec,n=25)

mat <- dplyr::select(top,-otu) %>% as.matrix(); rownames(mat) <- top$otu; storage.mode(mat) <- "numeric"

label_df <- taxonomy %>% dplyr::mutate(label = ifelse(!is.na(Genus) & Genus != "", Genus, ifelse(!is.na(Species) & Species != "", Species, OTU))) %>% dplyr::select(OTU, label)
rowlab <- tibble::deframe(label_df)[rownames(mat)]
rowlab[is.na(rowlab)] <- rownames(mat)[is.na(rowlab)]
dup <- duplicated(rowlab); rowlab[dup] <- paste0(rowlab[dup], "_", seq_len(sum(dup)))
rownames(mat) <- rowlab

mat <- mat[,sort(colnames(mat),method="radix")]

out_path <- file.path("plots", sub("\\.csv$","_top25_otus_by_trajectory.png", basename(clust_file)))
pheatmap::pheatmap(mat, scale="row", cluster_cols=FALSE, cluster_rows=TRUE, angle_col=45, fontsize_row=7, fontsize_col=10, filename=out_path)
cat("Saved:", out_path, "\n")



Warning message in dplyr::inner_join(clust_i, clust_f, by = "parent"):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 1 of `x` matches multiple rows in `y`.
ℹ Row 1 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship =
  "many-to-many"` to silence this warning.”


Saved: plots/otu_cluster_assignments_PAM_JSD_5x2_top25_otus_by_trajectory.png 


In [ ]:
# This generate a map of the forests using manuallly fetched coordinates. I could not use the authors' coordinates, some landed me in the sea in both formats.
system(".venv_py/bin/python generate_forests_map.py", intern = TRUE)

[1] ">>> STARTED"                                                      
[2] "Working directory: /home/lambi/Desktop/studia/ModZlo/FinalProject"
[3] "Saved: plots/forest_locations_static_map_zoomed2x.png"

In [ ]:
# This script generates a correlation between forest coordinates (latitude, longitude) and clusters

clust  <- read_csv("computed_data/clusterings/otu_cluster_assignments_PAM_JSD_5x2.csv", show_col_types = FALSE) %>% mutate(sampleid = sub("^X", "", sampleid))
meta   <- read.delim("initial_data/metadata_Time0D-7D-4M_May2022_wJSDpart-merged.csv")                       %>% mutate(sampleid = sub("^X", "", sampleid)) %>% select(sampleid, Location)
coords <- read.csv("initial_data/forest_coords.csv")

df <- clust %>% left_join(meta, by = "sampleid") %>% left_join(coords, by = "Location") %>%
       filter(timepoint %in% c("initial", "final"), !is.na(cluster), !is.na(Latitude), !is.na(Longitude))


# The amount of samples from diffrent locations is not equal, so we will use bootstrap (100 iters) sampling to balance the data.
set.seed(123); n_iter <- 100; n_per <- df %>% count(Location) %>% pull(n) %>% min()

boot_kw <- function(tp_df, coord) {
  res <- replicate(n_iter, {
    samp <- tp_df %>% group_by(Location) %>% slice_sample(n = n_per) %>% ungroup()
    if (n_distinct(samp$cluster) < 2) c(kw_stat = NA, p = NA)
    else with(kruskal.test(samp[[coord]] ~ factor(samp$cluster)), c(kw_stat = statistic, p = p.value))
  })
  res <- as.data.frame(t(res)); names(res) <- c("kw_stat", "p")
  mutate(res, timepoint = tp_df$timepoint[1], coord = coord)
}

res <- bind_rows(
  boot_kw(filter(df, timepoint == "initial"), "Latitude"),
  boot_kw(filter(df, timepoint == "initial"), "Longitude"),
  boot_kw(filter(df, timepoint == "final"),   "Latitude"),
  boot_kw(filter(df, timepoint == "final"),   "Longitude")
)

print(res %>% group_by(timepoint, coord) %>%
        summarise(mean_kw = mean(kw_stat, na.rm = TRUE),
                  mean_p  = mean(p,       na.rm = TRUE),
                  p_lt_0.05 = mean(p < 0.05, na.rm = TRUE),
                  .groups = "drop"))

plot_data <- df %>% group_by(Location, timepoint) %>% slice_sample(n = n_per) %>% ungroup()
box_plot  <- function(coord, tp)
  ggboxplot(filter(plot_data, timepoint == tp), x = "cluster", y = coord,
            fill = "cluster", palette = "Set2") +
  theme_minimal(base_size = 12) +
  labs(title = paste(coord, "by", tp, "cluster"), x = paste(tp, "cluster"), y = coord) +
  theme(legend.position = "none")


g <- gridExtra::arrangeGrob(
        box_plot("Latitude",  "initial"),
        box_plot("Latitude",  "final"),
        box_plot("Longitude", "initial"),
        box_plot("Longitude", "final"),
        nrow = 2)

ggsave("plots/geo_cluster_correlation_combined.png", plot = g, width = 14, height = 10)
cat("Saved: plots/geo_cluster_correlation_combined.png\n")



# A tibble: 4 × 5
  timepoint coord     mean_kw   mean_p p_lt_0.05
  <chr>     <chr>       <dbl>    <dbl>     <dbl>
1 final     Latitude   17.7   0.000255      1   
2 final     Longitude   0.794 0.494         0.01
3 initial   Latitude   20.8   0.00286       0.99
4 initial   Longitude   5.86  0.293         0.15
Saved: plots/geo_cluster_correlation_combined.png
